# Cassandra

In [ ]:
"""
cqlsh

CREATE KEYSPACE movie_space WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};
use movie_space;

CREATE TABLE movie_table(movieid int PRIMARY KEY, titles text, genre text);
CREATE TABLE rating_table(userid int, movieid int, rating double, timestamps bigint, primary key((userid), movieid));

COPY movie_table(movieid, titles, genre) FROM 'C:\data\ml-latest-small/movies.csv' WITH HEADER = true;
COPY rating_table(userid, movieid, rating, timestamps) FROM 'C:\data\ml-latest-small/ratings.csv' WITH HEADER = true;

SELECT * from movie_space.rating_table LIMIT 20;
INSERT INTO movie_space.rating_table (userid, movieid, rating) VALUES (1,2,3);
select * from movie_space.rating_table WHERE userid =1 LIMIT 20;
DELETE FROM movie_space.rating_table where userid = 1;
select * from movie_space.rating_table WHERE userid =1;
select * from movie_space.rating_table WHERE userid=23 ORDER BY movieid ASC LIMIT 30;
"""









"""
DESCRIBE KEYSPACES;
DESCRIBE TABLES;
nodetool status                      :provide info & summary about cluster
nodetool info                          :cache for high memory consumption
nodetool tpstats           :statistics about active,pending & completed tasks for each stage of cassandra ops
nodetool cfstats movie_space.rating_table

ALTER KEYSPACE movie_space WITH REPLICATION = { 'class' : 'NetworkTopologyStrategy','datacenter1' : 3 };
DESCRIBE SCHEMA;
DROP keyspace movie_space;

"""




"""
alter table rating_table drop rating;
alter table rating_table add rating2 text;
nodetool netstats                   :network info for the node
"""

In [1]:
#import and config pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import length
from matplotlib import pyplot as plt
import sys
import gc
import numpy as np
import pandas as pd
from collections import Counter

spark = SparkSession.builder.\
    config("spark.executor.memory", "4g")\
                  .master('local[*]') \
    .appName('project').getOrCreate()
sc = spark.sparkContext


In [2]:
# Creating PySpark SQL Context
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [3]:
from cassandra.cluster import Cluster

cluster = Cluster()
session = cluster.connect()

session = cluster.connect('movie_space')
#session.execute('USE  movie_space')

rows = session.execute('SELECT * from rating_table')
movie_ids = []
user_ids = []
ratings = []
for user_row in rows:
    #print(user_row.rating)
    #print(user_row.movieid)
    #print(user_row.userid)
    #print(user_row)
    movie_ids.append(user_row.movieid)
    user_ids.append(user_row.userid)
    ratings.append(user_row.rating)
#print("movie_ids", movie_ids)
print("average of ratings: ", np.average(ratings))
print("unique movie_ids: ",len(np.unique(movie_ids)))
print("unique user_ids:",len(np.unique(user_ids)))
z = Counter(movie_ids)
print("get No. of movie_id=365: ",z.get(356))
#print("movie_ids and total count of each: ",z)


    



average of ratings:  3.501556983616962
unique movie_ids:  9724
unique user_ids: 610
get No. of movie_id=365:  329


In [8]:
#rows = session.execute("CREATE KEYSPACE visi WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};")
#session.execute('USE  visi')

In [4]:
#cassandra and pandas
query = 'SELECT * from rating_table'
df = pd.DataFrame(list(session.execute(query)))

In [5]:
df.head(5)

,userid,movieid,rating,timestamps
0,23,6,4.0,1107342267
1,23,29,4.0,1107341574
2,23,32,3.5,1107341750
3,23,50,4.0,1107163741
4,23,58,3.0,1107164183


In [6]:
df.tail(3)

,userid,movieid,rating,timestamps
100833,103,140715,4.5,1440773817
100834,103,148626,5.0,1477364363
100835,103,168250,3.0,1522172079


In [7]:
df.dtypes

userid          int64
movieid         int64
rating        float64
timestamps      int64
dtype: object

In [8]:
df.describe()

,userid,movieid,rating,timestamps
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09
